## Semantic Kernel - Chat On your SQL Data

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#r "nuget: System.Data.SqlClient, 4.8.3"
#r "nuget: Microsoft.Extensions.Configuration, 6.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 6.0.0"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.ComponentModel;
using System.Text.Json.Serialization;
using System.Data.SqlClient;
using Microsoft.Extensions.Configuration;


Installed Packages Microsoft.Extensions.Configuration, 6.0.0 Microsoft.Extensions.Configuration.Json, 6.0.0 Microsoft.SemanticKernel, 1.23.0 System.Data.SqlClient, 4.8.3

In [2]:
var config = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/appSettings.json")
    .Build();

var username = config["SQL:Username"];
var password = config["SQL:Password"];
var server = config["SQL:Server"];
var database = config["SQL:Database"];

Console.WriteLine((username))

sqladmin


## Class to Retrieve DateTime with Semantic Kernel

In [3]:
public class TimeInformation
    {
        [KernelFunction]
        [Description("Retrieves the current time in UTC.")]
        public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
    }

In [18]:
using System;
using System.ComponentModel;
using System.Data.SqlClient;
using System.Text;
using Microsoft.SemanticKernel;

public class SQLPlugIn
{
    private readonly string _connectionString;

    public SQLPlugIn()
    {
        var config = new ConfigurationBuilder()
        .SetBasePath(Directory.GetCurrentDirectory())
        .AddJsonFile("config/appsettings.json")
        .Build();

        var username = config["SQL:Username"];
        var password = config["SQL:Password"];
        var server = config["SQL:Server"];
        var database = config["SQL:Database"];

        _connectionString = $"Server=tcp:{server};Initial Catalog={database};Persist Security Info=False;User ID=sqladmin;Password={password};MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;";
    }

    [KernelFunction, Description("Get list of tables in the database")]
    public string GetTables()
    {
        var query = "SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'";
        return ExecuteQuery(query);
    }

    private string ExecuteQuery(string query)
    {
        var sb = new StringBuilder();
        sb.Append("<table><tr><th>Schema</th><th>Table</th></tr>");

        using (var connection = new SqlConnection(_connectionString))
        using (var command = new SqlCommand(query, connection))
        {
            connection.Open();
            using (var reader = command.ExecuteReader())
            {
                while (reader.Read())
                {
                    sb.Append($"<tr><td>{reader["TABLE_SCHEMA"]}</td><td>{reader["TABLE_NAME"]}</td></tr>");
                }
            }
        }

        sb.Append("</table>");
        return sb.ToString();
    }

    [KernelFunction, Description("Get list of schema for a specific table")]
    public string GetTableSchema(string tableName)
    {
        var query = $"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{tableName}'";
        var sb = new StringBuilder();
        sb.Append("<table><tr><th>Column Name</th><th>Data Type</th></tr>");

        using (var connection = new SqlConnection(_connectionString))
        using (var command = new SqlCommand(query, connection))
        {
            connection.Open();
            using (var reader = command.ExecuteReader())
            {
                while (reader.Read())
                {
                    sb.Append($"<tr><td>{reader["COLUMN_NAME"]}</td><td>{reader["DATA_TYPE"]}</td></tr>");
                }
            }
        }

        sb.Append("</table>");
        return sb.ToString();
    }
    [KernelFunction, Description("Get run select statement database query")]
    public string RunSelectQuery(string query)
    {
        var sb = new StringBuilder();
        sb.Append("<table><tr>");

        using (var connection = new SqlConnection(_connectionString))
        using (var command = new SqlCommand(query, connection))
        {
            connection.Open();
            using (var reader = command.ExecuteReader())
            {
                // Add column headers
                for (int i = 0; i < reader.FieldCount; i++)
                {
                    sb.Append($"<th>{reader.GetName(i)}</th>");
                }
                sb.Append("</tr>");

                // Add rows
                while (reader.Read())
                {
                    sb.Append("<tr>");
                    for (int i = 0; i < reader.FieldCount; i++)
                    {
                        sb.Append($"<td>{reader[i]}</td>");
                    }
                    sb.Append("</tr>");
                }
            }
        }

        sb.Append("</table>");
        return sb.ToString();
    }


}


In [19]:
//Create Kernel builder
var builder = Kernel.CreateBuilder();
// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
builder.Plugins.AddFromType<TimeInformation>();
builder.Plugins.AddFromType<SQLPlugIn>();
var kernel = builder.Build();


#pragma warning disable SKEXP0001

OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

In [20]:
var ask = "What is the current time in UTC?";
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

The current time in UTC is Friday, 20 June 2025, 14:32:36.


In [21]:
var ask = "What are the tables in my database?";
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

The tables in your database are:

1. Capabilities
2. Incidents
3. Units
4. UnitCapabilities
5. IncidentNotes

If you need details about any specific table or want to see their contents, let me know!


In [22]:
var ask = "What are the columns of the table 'Capabilities'?";
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

The table 'Capabilities' has the following columns:
- CapabilityCode (nvarchar)
- Description (nvarchar)


In [25]:
var ask = "What are the Capabilities that are available?";
var result = await kernel.InvokePromptAsync(ask, new(openAIPromptExecutionSettings));

Console.WriteLine(result);

The available Capabilities are:

1. DRN - Drone Operator
2. FIRE - Fire Response
3. K9 - Canine Unit
4. MED - Medical Response
5. NEG - Negotiator
6. SWAT - Special Weapons and Tactics

If you need more details about any specific capability, let me know!


In [27]:
var history = "";

Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments

    // Process the user message and get an answer
    var answer = await kernel.InvokePromptAsync(input, new(openAIPromptExecutionSettings));

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    
    // Show the response
    Console.WriteLine(result);
};

In [30]:
Chat("What are the Capabilities do Units have?").Wait();


User: What are the Capabilities do Units have?
AI: Units have the following Capabilities:

- DRN: Drone Operator
- FIRE: Fire Response
- K9: Canine Unit
- MED: Medical Response
- NEG: Negotiator
- SWAT: Special Weapons and Tactics

These capabilities come from the capabilities assigned to units in your system. If you need the capabilities for a specific unit, please provide its identifier.



In [31]:
Chat("Who is currently available").Wait();


User: Who is currently available
AI: The following units are currently available:

1. UNIT-011 in Area-10 (Status: Idle)
2. UNIT-014 in Area-4 (Status: Idle)
3. UNIT-020 in Area-1 (Status: Idle)
4. UNIT-016 in Area-2 (Status: Idle)
5. UNIT-017 in Area-2 (Status: Idle)

If you need more details or want to see their locations, let me know!



In [32]:
Console.WriteLine("Chat history:");
Console.WriteLine(history);

Chat history:

User: What are the Capabilities that are available?
AI: Here are the Capabilities available in this environment:

1. **Answer questions using knowledge up to June 2024.**
2. **Retrieve and interact with external data via plugins/functions:**
   - Get current UTC time.
   - Explore, query, and retrieve data from databases (list tables, view table schemas, run select queries, etc.).
   - Use multi-tool operations to run several functions simultaneously.

If you have a specific task in mind or want details about using a particular function or plugin, please let me know!

User: What are the Capabilities do Units have?
AI: Units have the following Capabilities:

- DRN: Drone Operator
- FIRE: Fire Response
- K9: Canine Unit
- MED: Medical Response
- NEG: Negotiator
- SWAT: Special Weapons and Tactics

These capabilities come from the capabilities assigned to units in your system. If you need the capabilities for a specific unit, please provide its identifier.

User: Who is cur